In [1]:
try:
    import jpype
except:
    import jpype
from konlpy.tag import Kkma
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np
import re
import pandas as pd
import csv

In [2]:
sentences = ['/ 조선 DB “ 신용카드 사용 액이 소득 공제를 받을 수 있는 요건에 못 미치니까 아예 사용 액을 신고하지 않았던 겁니다.', '” 김 상조 공정거래위원장 후보자가 인사 검증 과정에서 신용카드 사용 액을 밝히지 않은 이유를 이렇게 설명했다.', '김 후보자는 카드 사용 액이 총소득의 4분의 1을 넘어야 하는 소득 공제 요건을 충족하지 못한 것이다.', '월급쟁이라면 응당 카드 소득 공제를 받는 것으로 생각하지만 김 후보자처럼 공제를 포기하거나 받지 못하는 사람들도 적지 않다.', '게다가 고 소득자를 중심으로 올해부터 카드 소득 공제 혜택을 줄일 예정이라 카드 공제 받기가 더 까다로 워 진다.', '11일 국세 통계 연보에 따르면, 2015년 기준으로 1733만명인 근로 소득자 중에서 카드 소득 공제를 받은 사람은 49.4% 인 856만명으로 절반에 못 미쳤다.', '그중 소득세가 면세( 免稅) 돼 세금을 내지 않은 257만명을 제외하고 실제로 세금을 낸 사람 중에서 카드 소득 공제를 받은 사람은 599만명으로 전체 월급쟁이의 34.5% 였다.', '◇ 연 봉 5000만원 대가 84% 로 카드 공제 가장 많이 받아 신용카드 소득 공제를 받으려면 총소득의 4분의 1 이상을 카드로 써야 한다.', '4분의 1 문턱에서 1원이라도 모자라면 한 푼도 공제 받을 수 없다.', '수억원 대의 고소득 자일 수록 카드로 소득의 4분의 1 이상을 쓰기 쉽지 않기 때문에 소득이 높을수록 공제를 못 받는 경우가 늘어난다.', '2015년 기준으로 근로 소득세를 내는 사람 중에서 카드 소득 공제를 받는 비율이 가장 많은 소득 구간은 연봉 5000만 ~6000 만원이었다.', '전체의 84% 가 혜택을 봤다.', '연봉 6000만 ~8000 만원 사이에서도 83%, 8000만 ~1 억원 구간은 79% 가 공제를 받았다.', '반면 1억 ~2 억원 사이에서는 공제 받은 사람 비율이 68% 로 줄었고, 2억 ~3 억원인 사람은 31% 만 카드 공제를 받은 것으로 나타났다.', '국세청 관계자는 “ 월급쟁이가 카드를 쓰는 용도가 다들 엇비슷해서 고소득 자라고 해서 카드 사용 액이 월등 하게 늘어나지는 않기 때문에 소득이 높을수록 ‘4 분의 1 문턱’ 을 넘기가 어려워지는 경향이 있다” 고 말했다.', '또 고 소득자들이 외국에 나가 카드를 많이 사용하지만, 해외에서의 카드 사용 액을 일절 공제해 주지 않는 점도 영향이 있는 것으로 분석된다.', '◇ 소득 낮은 부부는 한 사람 명의의 카드 써야 유리 소득이 연 5000만원에 못 미치는 서민층에서도 카드 공제를 받는 사람들의 비율이 낮아 지는 경향을 보인다.', '연봉 4000만원 대는 81%, 3000만원 대는 73%, 2000만원 대는 55% 만 카드 소득 공제를 받은 것으로 나타났다.', '이런 현상이 나타나는 이유는 우선 저소득층일수록 허리띠를 졸라매는 경향이 있어 ‘4 분의 1 문턱’ 을 넘지 못하는 경우가 상당하기 때문인 것으로 추정된다.', '또 소득이 넉넉하지 않은 맞벌이의 경우 한 사람 명의로 카드를 발급 받아 한 사람의 카드 사용 액이 ‘4 분의 1 문턱’ 을 넘도록 몰아주고, 나머지 배우자는 카드 소득 공제를 포기하는 절세 노하우를 보여주는 사례도 많다 고 국세청 관계자들은 설명했다.', '서울지역의 한 세무사는 “ 부부가 각자 명의로 카드를 쓰다가 둘 다 ‘4 분의 1 문턱’ 을 못 넘겨 안 내도 될 세금을 더 내는 경우가 있으니 주의해야 한다” 고 말했다.', '2015년 기준으로 카드 소득 공제를 받은 599만명은 평균 245만원을 공제 받은 것으로 나타났다.', '300만원인 공제한도를 채우지 못한 사람들이 적지 않다는 뜻이다.', '카드 소득 공제는 4분의 1 문턱을 초과하는 액수에 대해 신용카드는 15%, 체크카드는 30%에 해당하는 액수를 300만원 한도로 공제하는 방식이다.', '전통시장에서 결제한 금액과 대중 교통비에 대해서는 각각 100 만원씩 공제 한도가 더 늘어난다.', '◇ 연 봉 1억 2000만원 이상은 올해부터, 7000만원 이상은 내년부터 공제한도 축소 신용카드 소득 공제는 올해부터 순차적으로 한도가 줄어들게 되므로 관련 제도 변화를 유심히 살펴볼 필요가 있다.', '고소득 자의 공제 폭을 줄여 실질적인 근로 소득세 증세( 增稅) 가 예고돼 있다.', '우선 연소득 1억 2000만원이 넘는 월급쟁이는 올해 사용 분부터 공제한도가 기존 300만원에서 200만원으로 줄어든다.', '내년 초 연말 정산 때 돌려받는 세금이 수십만원 정도 줄어들거나, 토해 내는 세금이 수십만원 늘어난다는 뜻이다.', '연봉 7000만 ~1 억 2000만원 근로자의 공제 한도는 내년 사용 분부터 300만원에서 250만원으로 축소될 전망이다.', '연봉 7000만 ~1 억 2000만원인 근로자에 대해 정부는 2019년부터 한도를 축소하기로 했지만 지난해 국회가 1년 앞당겨 실시하기로 결정했다.']
words = ['조선 신용카드 사용 액 소득 공제 수 요건 못 아예 사용 액 신고', '김 상조 공정 거래 위원장 후보자 인사 검증 과정 신용카드 사용 액 이유 설명', '김 후보자 카드 사용 액 소득 소득 공제 요건 충족 못 것', '월급쟁이 라면 응당 카드 소득 공제 것 생각 김 후보자 공제 포기 거나 사람 적지', '게다가 고 소득 중심 올해 카드 소득 공제 혜택 예정 카드 공제 더 워', '국세 통계 연보 기준 근로 소득 중 카드 소득 공제 사람 인 절반 못', '소득세 면세 세금 내지 제외 실제 세금 사람 중 카드 소득 공제 사람 전체 월급쟁이', '연 봉 대가 로 카드 공제 가장 신용카드 소득 공제 소득 이상 카드', '문턱 푼 공제 수', '억원 대의 소득 자일 수록 카드 소득 이상 쓰기 때문 소득 공제 못 경우', '기준 근로 소득세 사람 중 카드 소득 공제 비율 가장 소득 구간 연봉 만원', '전체 혜택', '연봉 만원 사이 억원 구간 공제', '반면 억원 사이 공제 사람 비율 로 억원 사람 만 카드 공제 것', '국세청 관계자 월급쟁이 카드 용도 소득 카드 사용 액 월등 때문 소득 분 문턱 경향 고 말', '또 고 소득 외국 카드 사용 해외 카드 사용 액 일절 공제 주지 점도 영향 것 분석', '소득 부부 사람 명의 카드 유리 소득 연 못 서민 층 카드 공제 사람 비율 경향', '연봉 만 카드 소득 공제 것', '현상 이유 우선 소득 층 허리띠 졸라 경향 분 문턱 경우 때문 것 추정', '또 소득 맞벌이 경우 사람 명의 카드 발급 사람 카드 사용 액 분 문턱 나머지 배우자 카드 소득 공제 포기 절세 노하우 사례 고 국세청 관계자 설명', '지역 세무사 부부 각자 명의 카드 둘 분 문턱 못 안 세금 더 경우 주의 고 말', '기준 카드 소득 공제 평균 공제 것', '한도 못 사람 적지 뜻', '카드 소득 공제 문턱 초과 액수 대해 신용카드 체크카드 해당 액수 한도 공제 방식', '전통 시장 결제 금액 대중 교통비 대해 각각 만원 공제 한도 더', '연 봉 이상은 올해 이상은 내년 공제 한도 축소 신용카드 소득 공제 올해 순차 한도 관련 제도 변화 유심 필요', '소득 자의 공제 폭 실질 근로 소득세 증세 예고', '우선 소득 월급쟁이 올해 사용 분 한도 기존', '내년 초 연말 정산 때 세금 만원 정도 토 세금 만원 뜻', '연봉 억 근로자 공제 한도 내년 사용 분 축소 전망', '연봉 억 근로자 대해 정부 한도 축소 지난해 국회 실시 결정']

In [3]:
def summary(sentences, words):
    ## GraphMatrix
    tfidf = TfidfVectorizer()
    cnt_vec = CountVectorizer()
    sentence_graph = []

    # build_sent_graph
    tfidf_mat = tfidf.fit_transform(words).toarray()
    sentence_graph = np.dot(tfidf_mat, tfidf_mat.T)

    # build_words_graph
    data = cnt_vec.fit_transform(words).toarray()
    cnt_vec_mat = normalize(data, axis=0)
    vocab = cnt_vec.vocabulary_
    words_graph = np.dot(cnt_vec_mat.T, cnt_vec_mat)
    idx2word = {vocab[word] : word for word in vocab}
    
    ## Rank
    # sentence graph
    A = sentence_graph
    d = 0.85
    matrix_size = A.shape[0]

    for id in range(matrix_size):
        A[id, id] = 0 # diagonal 부분을 0으로
        link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
        if link_sum != 0:
                A[:, id] /= link_sum
        A[:, id] *= -d
        A[id, id] = 1

    B = (1-d) * np.ones((matrix_size, 1))
    ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b

    sentence_graph_idx = {idx: r[0] for idx, r in enumerate(ranks)}

    sorted_sentence_rank_idx = sorted(sentence_graph_idx, key=lambda k: sentence_graph_idx[k], reverse=True)

    # word graph
    A = words_graph
    d = 0.85
    matrix_size = A.shape[0]

    for id in range(matrix_size):
        A[id, id] = 0 # diagonal 부분을 0으로
        link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
        if link_sum != 0:
                A[:, id] /= link_sum
        A[:, id] *= -d
        A[id, id] = 1

    B = (1-d) * np.ones((matrix_size, 1))
    ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b

    words_graph_idx = {idx: r[0] for idx, r in enumerate(ranks)}

    sorted_words_rank_idx = sorted(words_graph_idx, key=lambda k: words_graph_idx[k], reverse=True)

    # summarize
    sent_num=3
    summary = []
    index=[]

    for idx in sorted_sentence_rank_idx[:sent_num]:
        index.append(idx)

    index.sort()
    for idx in index:
        summary.append(sentences[idx])

    # keywords
    word_num=10
    keywords = []
    index=[]

    for idx in sorted_words_rank_idx[:word_num]:
        index.append(idx)
        
    #index.sort()
    for idx in index:
        keywords.append(idx2word[idx])

    count = 0

    for row in summary:
        print(row)
        print()
        count = count + 1
        if(count > 3):
            break

    print('keywords :',keywords)
    return keywords

In [4]:
def text_processing(start, end):
    table = dict()
    with open('f:/eunjin/OSSW-Project/Sentiment-analysis/polarity.csv', 'r', -1, 'utf-8') as polarity:
        next(polarity)
        
        for line in csv.reader(polarity):
            key = str()
            for word in line[0].split(';'):
                key += word.split('/')[0]
            table[key] = {'Neg': line[3], 'Neut': line[4], 'Pos': line[6]}

    columns=['negative', 'neutral', 'positive']
    df = pd.DataFrame(columns=columns)

    file_stop_word = open('f:/eunjin/OSSW-Project/Sentiment-analysis/stop_words_file.txt', 'r', -1, 'utf-8')
    stop_word = file_stop_word.read()
    stop_word_list = []
    negative_list = []
    neutral_list = []
    positive_list = []
    for word in stop_word.split(','):
        if word not in stop_word_list:
            stop_word_list.append(word)
            file_stop_word.close()

    for i in range(start, end):
        words = str(sentences)
        hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
        words = hangul.sub('', words)
        words_list = []
        for i in words:
            if i not in stop_word_list:
                words_list.append(i)

    negative = 0
    neutral = 0
    positive = 0

    for word in words_list:
        if word in table:
            negative += float(table[word]['Neg'])
            neutral += float(table[word]['Neut'])
            positive += float(table[word]['Pos'])
  
    negative_list.append(negative)
    neutral_list.append(neutral)
    positive_list.append(positive)

    df['negative'] = negative_list
    df['neutral'] = neutral_list
    df['positive'] = positive_list

    return df

In [5]:
def sentiment_analysis():
    df = text_processing(0,366)

    df.to_csv('./result.csv', index=False)

    ds = pd.read_csv('./result.csv')
    ds == ds.values.max()
    ids, cols = np.where(ds == ds.values.max())
    list(zip(ids, cols))
    sen = [ds.columns[c] for c in cols]
    return sen

In [6]:
sentiment_analysis()

['positive']

In [7]:
summary(sentences, words)

◇ 연 봉 5000만원 대가 84% 로 카드 공제 가장 많이 받아 신용카드 소득 공제를 받으려면 총소득의 4분의 1 이상을 카드로 써야 한다.

2015년 기준으로 근로 소득세를 내는 사람 중에서 카드 소득 공제를 받는 비율이 가장 많은 소득 구간은 연봉 5000만 ~6000 만원이었다.

연봉 4000만원 대는 81%, 3000만원 대는 73%, 2000만원 대는 55% 만 카드 소득 공제를 받은 것으로 나타났다.

keywords : ['소득', '공제', '카드', '사람', '사용', '한도', '경우', '신용카드', '문턱', '설명']


['소득', '공제', '카드', '사람', '사용', '한도', '경우', '신용카드', '문턱', '설명']